In [1]:
!pip install requests
!pip install tsplib95 # modulo para las instancias del problemas del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.6 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [2]:
import urllib.request
import tsplib95
import math
import random

In [3]:

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

In [4]:
problem = tsplib95.load(file)

In [5]:

Nodos = list(problem.get_nodes())

In [6]:

Aristas = list(problem.get_edges())

In [24]:

problem.get_weight(3,2)

11

In [14]:
#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

In [18]:

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [19]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)

Mejor solución: [0, 5, 28, 23, 18, 3, 30, 17, 29, 24, 20, 27, 32, 15, 16, 2, 34, 33, 26, 10, 39, 41, 8, 7, 37, 36, 35, 22, 25, 11, 12, 38, 40, 21, 9, 1, 4, 6, 31, 19, 14, 13]
Distancia     : 3541


In [20]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 3541
Distancia Mejor Solucion Local: 3309


In [21]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

En la iteracion  34 , la mejor solución encontrada es: [0, 30, 29, 8, 9, 21, 24, 40, 41, 25, 11, 12, 18, 13, 5, 26, 2, 28, 27, 3, 4, 6, 1, 7, 32, 38, 22, 39, 23, 10, 19, 14, 16, 15, 37, 17, 31, 36, 35, 20, 33, 34]
Distancia     : 1683


In [32]:
def multi_arranque(problem, arranques):
    mejores_soluciones = []

    for _ in range(arranques):
        solucion = busqueda_local(problem)
        mejores_soluciones.append(solucion)

    mejor_solucion = min(mejores_soluciones, key=lambda x: distancia_total(x, problem))
    return mejor_solucion

In [33]:
#intentar la busqueda local n con cinco arranques aleatorios que empiezan en el 0
arranques = 5  # Número de arranques a realizar
mejor_solucion = multi_arranque(problem, arranques)
print("La mejor solución encontrada es:", mejor_solucion)

En la iteracion  33 , la mejor solución encontrada es: [0, 20, 33, 34, 9, 23, 40, 24, 21, 8, 18, 26, 5, 6, 4, 2, 27, 3, 1, 7, 17, 36, 35, 31, 32, 30, 29, 10, 12, 11, 25, 41, 39, 22, 38, 28, 13, 19, 14, 16, 15, 37]
Distancia     : 1819
En la iteracion  30 , la mejor solución encontrada es: [0, 32, 34, 33, 20, 17, 37, 15, 14, 1, 29, 21, 24, 40, 8, 10, 18, 26, 5, 6, 4, 3, 27, 2, 28, 30, 38, 22, 39, 9, 23, 41, 25, 11, 12, 13, 19, 16, 36, 35, 31, 7]
Distancia     : 1677
En la iteracion  48 , la mejor solución encontrada es: [0, 1, 13, 19, 16, 37, 17, 36, 35, 31, 27, 2, 8, 41, 23, 40, 24, 21, 9, 10, 25, 11, 12, 18, 26, 20, 33, 34, 32, 28, 30, 38, 22, 39, 29, 7, 15, 14, 5, 6, 4, 3]
Distancia     : 1786
En la iteracion  45 , la mejor solución encontrada es: [0, 20, 33, 34, 32, 2, 4, 12, 11, 18, 26, 5, 13, 19, 14, 16, 15, 37, 7, 1, 6, 3, 31, 17, 36, 35, 38, 22, 39, 21, 24, 40, 9, 23, 41, 25, 10, 8, 29, 30, 28, 27]
Distancia     : 1585
En la iteracion  39 , la mejor solución encontrada es: [0, 1